In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
SOURCE_FILE_FOLDER = 'Q:\OdooMigration\Accounting\costCenterInputs\\'
#SOURCE_FILE_FOLDER = 'C:\\Users\\bmeie\\Documents\\Accounting\\costCenterInputs\\'
import datetime
now = datetime.datetime.now()
print(f"Report generated at {now.strftime('%H:%M:%S on %A, %B the %dth, %Y')}.")

Report generated at 13:48:21 on Thursday, September the 09th, 2021.


# Allocation of Costs To Cost Centers
## Costs to be allocated:
- Depreciation
- Building Taxes
- Other Expenses

## Buildings/ Departments/ Cost Centers
**Times are generated by costing program based on a reasonable model**
**Changes to times require chagnes to rates**
**Costs at uncosted work centers aren't free, they are included in the burdend rate of the driver**
- Moravia (MOR)
    - Sheetmetal (SM)
        - Cutting (CUT); Driver is cutting time
        - Bending, welding, pemming (OTH); Driver is bend or weld time.  This includes welding of CAC pipes
        - CAC tubing (TUB); Driver is time.  Includes bend, cut, debur beading, etc.
- New Butler Road (NBR)
- Velocity (VEL)
    - Light assembly, HEX assembly (AS)
        - ASY; Driver is assembly time.
    - Aluminum (AL)
        - Machining, Robot welding, hand welding, leak testing (ALU); Driver is welding and machine hours
    
**Note:**  For simplification, light assembly is being treated as though it is in velocity.  The justification for this is that the jobs are similar, the capital equipment is small, the light assembly area is small, and both groups report to the same manager.

**Note:**  Due to similar nature of machines and work and the difficulty separating employees between tasks, the machining group and the Aluminum group are being treated as one cost center.  The driver is time, either robot weld, hand weld or machining.



## Buildings and Cost Center Information
**MOR:** Warehouse/ manufacturing area is divided into equal parts CUT, OTH, TUB and warehouse.

**NBR:** Warehouse/ manufacturing area is considered to be 100% warehouse.

**VEL:** 

- 20% AS
- 20% AL
- 40% Warehouse
- 20% SG&A (Quality)

In [2]:
def saveAsCsv(df, fname, folder=SOURCE_FILE_FOLDER):
    # Saves dataframe as a csv file
    # example call: saveAsCsv(buildDf, 'build.csv')
    df.to_csv(folder+f'{fname}',index=None)

In [3]:
def formatNums(dfInp):
    df=dfInp.copy()
    lst = list(df.columns)
    for c in lst:
        try:
            df[c] = df[c].astype(int).apply(lambda x: f'{x:,}')
        except:
            pass
    return df

In [4]:
buildDf     = pd.read_csv(SOURCE_FILE_FOLDER +'build.csv')  # Stores building total, office and work center areas
buildDeptDf = pd.read_csv(SOURCE_FILE_FOLDER +'buildDept.csv') # Defines department codes. 
                                                               # Building and department form primary key
deptCcDf    = pd.read_csv(SOURCE_FILE_FOLDER +'deptCc.csv') # Defines cost centers ID, code and areas. 
formatNums(buildDf)

,Building,bCode,TotAreaSqft,OfficeAreaSqft,WarehouseAreaSqft
0,MOR,1,"40,100","16,100","6,000"
1,NBR,2,"64,000","8,000","56,000"
2,VEL,3,"56,000","11,200","22,400"


In [5]:
df0 = deptCcDf.set_index('Department').join(buildDeptDf.set_index('Department')).reset_index()
df = df0.set_index('Building').join(buildDf[['Building', 'bCode']].set_index('Building')).reset_index()
df = df[['Building','Department', 'CostCenter','ccSqft','bCode','dCode','cCode']]
df['Code'] = 100*df['bCode']+10*df['dCode']+df['cCode']
df = df[['Building','Department', 'CostCenter','ccSqft','Code','bCode','dCode','cCode']]
df = df.sort_values(['Code']).reset_index(drop=True)
df['Desc']=df['Building']+'_'+df['Department']+'_'+df['CostCenter']
costCenterDf = df
formatNums(costCenterDf)

,Building,Department,CostCenter,ccSqft,Code,bCode,dCode,cCode,Desc
0,MOR,SM,CUT,"6,000",111,1,1,1,MOR_SM_CUT
1,MOR,SM,OTH,"6,000",112,1,1,2,MOR_SM_OTH
2,MOR,SM,TUB,"6,000",113,1,1,3,MOR_SM_TUB
3,VEL,AS,ASY,"11,200",311,3,1,1,VEL_AS_ASY
4,VEL,AL,ALU,"11,200",321,3,2,1,VEL_AL_ALU


In [6]:
buildCcAreaDf = df[['Building', 'Department', 'ccSqft']].groupby('Building')['ccSqft'].sum().reset_index()
buildDf = buildDf.set_index('Building').join(buildCcAreaDf.set_index('Building')).reset_index().fillna(0)
buildDf['AreaError'] = buildDf['TotAreaSqft'] - buildDf['OfficeAreaSqft'] - buildDf['WarehouseAreaSqft'] - buildDf['ccSqft']
#print('Confirm building totals are accurate:')
buildDf.rename(columns={'ccSqft':'CostCentersAreaSqft'},inplace=True)
buildDf.loc[:,['Building', 'AreaError']]

,Building,AreaError
0,MOR,0.0
1,NBR,0.0
2,VEL,0.0


In [7]:
# Add Building and Department Codes/ Description
bCodeDescDf = buildDf.loc[:,['Building', 'bCode']]
bCodeDescDf['Code']=bCodeDescDf['bCode']*100
bCodeDescDf.set_index('Code',inplace=True)
bCodeDescDf=bCodeDescDf[['Building']]
bCodeDescDf.columns=['Desc']

deptCodeDescDf = costCenterDf.loc[:,['Building', 'Department', 'bCode', 'dCode']]
deptCodeDescDf.drop_duplicates(inplace=True)
deptCodeDescDf['Code']=deptCodeDescDf['bCode']*100+deptCodeDescDf['dCode']*10
deptCodeDescDf=deptCodeDescDf.loc[:,['Department', 'Code']]
deptCodeDescDf.set_index('Code', inplace=True)
deptCodeDescDf.columns=['Desc']

## Code Descriptions

In [8]:
codeNotCcDf = pd.read_csv(SOURCE_FILE_FOLDER +'codeDesc.csv') # stores descriptions and codes for 
                                                              # non-cost center codes (building, etc.)
codeNotCcDf['Code']=codeNotCcDf['Code'].astype(int)
codeNotCcDf = codeNotCcDf.sort_values(['Code']).reset_index(drop=True)
codeNotCcDf.set_index('Code', inplace=True)
codeDescDf = pd.concat([costCenterDf[['Code','Desc']].set_index('Code'), bCodeDescDf, deptCodeDescDf, codeNotCcDf])
codeDescDf.sort_values('Code', inplace=True)
codeDescDf.reset_index(inplace=True)
codeDescDf.drop_duplicates(inplace=True)
codeDescDf.set_index('Code', inplace=True)
#saveAsCsv(codeDescDf.reset_index(),'codeDesc.csv')
codeDescDf

,Desc
Code,
-10,IntMoves
-5,IncludedInTaxSection
-2,ContraRevenue
-1,Product
0,RCPall
1,RCPman
2,ManEmployee
3,ExpTooling
4,ExpGas


## Area Splits
- Breaks apart Building Information and gets it into codes for allocation by area

In [9]:
def addDesc(outDf, descDf=codeDescDf):
    cList = outDf.columns.values.tolist()
    cols = ['Desc']+cList
    outDf=outDf.join(descDf)
    outDf = outDf[cols]
    return outDf

In [10]:
codeCcAreaDf = costCenterDf.copy()
codeCcAreaDf = codeCcAreaDf[['Code', 'ccSqft']]
codeCcAreaDf.columns = ['Code','area']

codeWhAreaDf = buildDf.copy()[['bCode','WarehouseAreaSqft']]
codeWhAreaDf['Code'] = codeWhAreaDf['bCode']*100+80
codeWhAreaDf=codeWhAreaDf[['Code','WarehouseAreaSqft']]
codeWhAreaDf.columns=['Code','area']

codeOfAreaDf = buildDf.copy()[['bCode','OfficeAreaSqft']]
codeOfAreaDf['Code'] = codeOfAreaDf['bCode']*100+90
codeOfAreaDf=codeOfAreaDf[['Code','OfficeAreaSqft']]
codeOfAreaDf.columns=['Code','area']

codeAreaDf = pd.concat([codeCcAreaDf, codeWhAreaDf, codeOfAreaDf])
codeAreaDf.sort_values(['Code']).reset_index(inplace=True)
codeAreaDf.set_index('Code', inplace=True)
addDesc(codeAreaDf)
#formatNums(addDesc(codeAreaDf)).sort_values('Code')

,Desc,area
Code,,
111,MOR_SM_CUT,6000
112,MOR_SM_OTH,6000
113,MOR_SM_TUB,6000
311,VEL_AS_ASY,11200
321,VEL_AL_ALU,11200
180,MORwh,6000
280,NBRwh,56000
380,VELwh,22400
190,MORoffice,16100


## Labor Splits
**source:**  'Work Centers 08 11 21.xlsx from Carla Richardson in e-mail sent on 8/12/21
**ID stripped and resorted:** Labor by WC 08 11 21

### Data is for pay period ending 8/7/21

Becasue the rates are similar, allocating labor costs based on cost center head count was selected.

**NOTE:** Lloyd Harlan is included in the AL/MAC department; Troy Boron is not included in any department.

In [11]:
codeHeadCountDf = pd.read_csv(SOURCE_FILE_FOLDER +'manHeadCount.csv')
codeHeadCountDf.sort_values(['Code']).reset_index(inplace=True)
codeHeadCountDf.set_index('Code', inplace=True)
addDesc(codeHeadCountDf)

,Desc,headCount
Code,,
111,MOR_SM_CUT,1.94
112,MOR_SM_OTH,12.94
113,MOR_SM_TUB,2.12
311,VEL_AS_ASY,13.00
321,VEL_AL_ALU,9.00


In [12]:
hcSM = codeHeadCountDf.loc[111,'headCount']+codeHeadCountDf.loc[112,'headCount']+codeHeadCountDf.loc[113,'headCount']
hcAS = codeHeadCountDf.loc[311,'headCount']
hcAL = codeHeadCountDf.loc[321,'headCount']

print('Total Head Counts')
print(f"Sheet Metal       Head Count: {hcSM:0.0f}")
print(f"Assembly          Head Count: {hcAS:0.0f}")
print(f"Aluminum          Head Count: {hcAL:0.0f}")
print(f"All Manufacturing Head Count: {hcSM+hcAS+hcAL:0.0f}")

Total Head Counts
Sheet Metal       Head Count: 17
Assembly          Head Count: 13
Aluminum          Head Count: 9
All Manufacturing Head Count: 39


## Allocation Method Definition
Deprciation is allocated based on infor in the **codeChildParent.csv** file
- Stores Code, Children, bases (equal, area or given allocation), and Parent
- Numbers are separated with a single space
- A code can have any number of children
- A code can have no or one Parent
- Codes that are not cost centers should have children or a parent

In [13]:
codeChildParentDf = pd.read_csv(SOURCE_FILE_FOLDER +'codeChildParent.csv')
# Stores Code, Children, bases (equal, area or given allocation), and Parent
# Numbers are separated with a single space
# A code can have any number of children
# A code can have no or one Parent
# Codes that are not cost centers should have children or a parent
codeChildParentDf['Children'].replace({np.nan:None}, inplace=True)
codeChildParentDf['Basis'].replace({np.nan:None}, inplace=True)
codeChildParentDf['Parent'].replace({np.nan:None}, inplace=True)
codeChildParentDf = codeChildParentDf.sort_values(['Code']).reset_index(drop=True)
children = []
for i in codeChildParentDf['Children']:
    if i is None:
        children.append(i)
    if isinstance(i,str):
        kids = [int(x) for x in i.split()]
        children.append(kids)
    if isinstance(i,float):
        kid = [int(i)]
        children.append(kid)
codeChildParentDf['Children'] = children
codeChildParentDf.set_index('Code',inplace=True)

codeChildParentViewDf=codeChildParentDf.copy()
codeChildParentViewDf.Parent.values
parents = codeChildParentViewDf.Parent.values
pList = []
for p in parents:
    try:
        intP=int(p)
        pList.append(str(intP))
    except:
        pList.append(None)
codeChildParentViewDf['Parent']=pList
print(f" AllRCP Children: {codeChildParentViewDf.loc[0,'Children']}")
addDesc(codeChildParentViewDf)

 AllRCP Children: [111, 112, 113, 311, 321, 180, 190, 280, 290, 380, 390]


,Desc,Children,Basis,Parent
Code,,,,
-10,IntMoves,None,None,None
-5,IncludedInTaxSection,None,None,None
-2,ContraRevenue,None,None,None
-1,Product,None,None,None
0,RCPall,"[111, 112, 113, 311, 321, 180, 190, 280, 290, ...",area,None
1,RCPman,"[111, 112, 113, 311, 321]",equal,None
2,ManEmployee,"[111, 112, 113, 311, 321]",headCount,None
3,ExpTooling,"[-1, 111, 113, 321]",.251 .514 .033 .203,None
4,ExpGas,"[111, 321, 900]",50 25 25,None


In [14]:
def getValue(Code, codeDf, cName):
    # codeDf is a dataframe with:
    # Code as the index
    # Value (examples inlude Area (in sqft) and headCount (in numEmployees) as a column
    return codeDf.loc[Code,cName]

In [15]:
def allocate(codeChildParentDf, dfInput, costName, codeAreaDf, codeHeadCountDf):
    # codeChildParentDf is used to allocate costs.  It is a dataframe with
    # Code as the index, and columns Childre, Basis and Parent
    # Children can be None, or a list of codes to allocate.
    # Basis can be:
    # equal (split equally between children)
    # area (split according to area)
    # manual (list of numbers representing, in order, the relative amount allocated to each child)
    # -- these numbers are normalized, so they don't need to add to one and can represent any measure.
    # Parent can be None, or the Code to allocate the cost
    # EVERY Code that is not a cost center must have children or a parent!
    #
    # dfInput contains the costs to be allocated
    # The Code is the index
    # costName is the name of the column for which costs maust be allocated
    # ChildAllocateAdj,  ChildAllocateAdjusted, ParentAllocateAdj and ParentAllocateAdjusted
    # are intermediate columns added to the df dataframe
    # ccCost is the cost that has been allocated to the cost centers
    # df is returned
    
    df = dfInput.copy()

    df['ChildAllocateAdj']=0
    for c in codeChildParentDf.index:
        kids = codeChildParentDf.loc[c,'Children']
        method = codeChildParentDf.loc[c,'Basis']
        #print(f"{c=}; {method=}; {kids=}")
        if kids != None:
            cont=True
            try:
                totAllocate = df.loc[c,costName]
            except:
                cont = False
            #print(f"{method=}")
            if cont==True:
                df.loc[c,'ChildAllocateAdj'] = -1*df.loc[c,costName]
                if method=='equal':
                    #print('In Equal')
                    perAllocate = totAllocate*1.0/len(kids)
                    #print(f"{perAllocate*len(kids)-1*df.loc[c,costName]=};{perAllocate=}")
                    for k in kids:
                        df.loc[k,'ChildAllocateAdj']=df.loc[k,'ChildAllocateAdj']+perAllocate
                if method=='area':
                    #print('In Area')
                    areas = []
                    for k in kids:
                        areas.append(getValue(k, codeAreaDf,method))
                    perAllocate=areas/sum(areas)*totAllocate
                    #print(f"{perAllocate.sum()-1*df.loc[c,costName]=};{perAllocate=}")
                    for k, adj in zip(kids, perAllocate):
                        df.loc[k,'ChildAllocateAdj']=df.loc[k,'ChildAllocateAdj']+adj
                if method=='headCount':
                    #print('In HeadCount')
                    headCounts=[]
                    for k in kids:
                        headCounts.append(getValue(k, codeHeadCountDf, method))
                    perAllocate=headCounts/sum(headCounts)*totAllocate
                    #print(f"{perAllocate.sum()-1*df.loc[c,costName]=};{perAllocate=}")
                    for k, adj in zip(kids, perAllocate):
                        df.loc[k,'ChildAllocateAdj']=df.loc[k,'ChildAllocateAdj']+adj
                    #print(f"{c=} {kids=}; {headCounts=}; {perAllocate=}")
                try:
                    floats = np.array(list(map(float,method.split())))
                    perAllocate = floats/sum(floats)*totAllocate
                    #print(f"{perAllocate.sum()-1*df.loc[c,costName]=};{perAllocate=}")
                    for k, adj in zip(kids, perAllocate):
                        df.loc[k,'ChildAllocateAdj']=df.loc[k,'ChildAllocateAdj']+adj
                    #print('In Custom')
                except:
                    pass
    df['ChildAdjusted'] = df[costName]+df['ChildAllocateAdj']

    # Allocate Parents -- MUST BE DONE AFTER CHILD ALLOCATIONS
    df['ParentAllocateAdj']=0.0
    for c in codeChildParentDf.index:
        parent = codeChildParentDf.loc[c,'Parent']
        if parent != None:
            cont=True
            try:
                totParentAllocate = df.loc[c,'ChildAdjusted']
            except:
                cont = False
            if cont==True:   
                df.loc[c,'ParentAllocateAdj'] = -1*totParentAllocate
                df.loc[parent,'ParentAllocateAdj']=df.loc[parent,'ParentAllocateAdj']+totParentAllocate
    df['ccCost'] = df['ChildAdjusted']+df['ParentAllocateAdj']
    
    return df

## Depreciation Allocation
**Source:** MAS500 Report

**File** Q:\OdooMigration\Accounting\depreciation.msg attachement: **Copy of GAAP Depreciation Comprehensive Depreciation Letter Size 1.xls**

Edited and saved into Depr_8_10_2021.csv

**DateRun:** 8/10/2021

In [16]:
def addcDescCol(outDf, descDf=codeDescDf):
    descDf = descDf.copy()
    descDf.reset_index(inplace=True)
    descDf.columns=['Code','cDesc']
    outDf = pd.merge(
        outDf,
        descDf,
        how="left",
        )
    return outDf

In [17]:
assetIdCodeDf = pd.read_csv(SOURCE_FILE_FOLDER +'assetIdCode.csv', converters={'AssetID': lambda x: str(x)})
# stores a link between asset IDs and thier Codes
depDf = pd.read_csv(SOURCE_FILE_FOLDER +'Depr_8_10_2021.csv', converters={'AssetID': lambda x: str(x)})
# see note above.  Contains AssetID, Desc, DeprYTD
depDf = depDf.set_index('AssetID').join(assetIdCodeDf.set_index('AssetID')).reset_index()

In [18]:
depDfView=depDf.copy()
depDfView = depDfView.loc[depDfView['DeprYTD']>0.01]
depDfView.sort_values(by=['Code','DeprYTD'],ascending=[True, False], inplace=True)
depDfView['DeprYTD']=depDfView['DeprYTD'].apply(lambda x : "{:,.2f}".format(x))
depDfView = addcDescCol(depDfView)
depDfView

,AssetID,Desc,DeprYTD,Code,cDesc
0,000481,Compressor at Moravia,"1,929.44",110,SM
1,000485,Oil Pump for Machine Maintenance,250.44,110,SM
2,000499,Tru Laser 2030 Fiber,"54,755.04",111,MOR_SM_CUT
3,000522,TruLaser 3030 Fiber - Upgarde,"33,535.23",111,MOR_SM_CUT
4,000502,TruPunch (S18),"30,041.31",111,MOR_SM_CUT
5,000500,Tru Laser - Liftmaster Shuttle,"14,901.00",111,MOR_SM_CUT
6,000504,TruPunch Sheet Master Compact Basic,"9,958.69",111,MOR_SM_CUT
7,000554,Laser 3030 Kits,"2,056.96",111,MOR_SM_CUT
8,000510,Punch Tooling - 43 Pieces,"1,665.61",111,MOR_SM_CUT
9,000503,Tank Pads - Moravia - for bulk holding tanks f...,"1,414.93",111,MOR_SM_CUT


In [19]:
############# MAKE SURE TO CHECK MONTHS_YTD!!!! ######################
MONTHS_YTD=7
depCodeDf = depDf.groupby('Code')['DeprYTD'].sum().reset_index()
depCodeDf.loc[len(depCodeDf.index)] = [190, 0]
depCodeDf['DeprAnnUnallocated']=12/MONTHS_YTD*depCodeDf['DeprYTD']
depCodeDf.set_index('Code',inplace=True)

In [20]:
depCodeDf = allocate(codeChildParentDf, depCodeDf, 'DeprAnnUnallocated', codeAreaDf, codeHeadCountDf)
formatNums(addDesc(depCodeDf).round())

,Desc,DeprYTD,DeprAnnUnallocated,ChildAllocateAdj,ChildAdjusted,ParentAllocateAdj,ccCost
Code,,,,,,,
0,RCPall,0,0,0,0,0,0
110,SM,"2,180","3,737","-3,737",0,0,0
111,MOR_SM_CUT,"154,871","265,493","1,246","266,739",0,"266,739"
112,MOR_SM_OTH,"79,697","136,623","1,246","137,869",0,"137,869"
113,MOR_SM_TUB,"28,247","48,424","1,246","49,670",0,"49,670"
180,MORwh,"6,310","10,817",0,"10,817","-10,817",0
200,NBR,"15,025","25,758","-25,758",0,0,0
280,NBRwh,"23,360","40,045","22,538","62,583","-62,583",0
290,NBRoffice,"1,340","2,296","3,220","5,516","-5,516",0


In [21]:
#print(f"{depCodeDf.loc[:,'ChildAllocateAdj'].sum().round(4)=}")
#print(f"{depCodeDf.loc[:,'ParentAllocateAdj'].sum().round(4)=}")

print(f"Sum of Child  Adjustment Check: {depCodeDf.loc[:,'ChildAllocateAdj'].sum().round(4)}")
print(f"Sum of Parent Adjustment Check: {depCodeDf.loc[:,'ParentAllocateAdj'].sum().round(4)}")

Sum of Child  Adjustment Check: 0.0
Sum of Parent Adjustment Check: 0.0


## Building Tax Allocation
**Source:**  Paper files from Carla

In [22]:
buildTaxDf = pd.read_csv(SOURCE_FILE_FOLDER +'buildTaxes.csv')
buildTaxDf.set_index('Code',inplace=True)
buildTaxDf.fillna(0,inplace=True)
buildTaxDf = allocate(codeChildParentDf, buildTaxDf, 'Total', codeAreaDf, codeHeadCountDf)
formatNums(addDesc(buildTaxDf))

,Desc,BuildName,School,City,County,NCpriv,Total,ChildAllocateAdj,ChildAdjusted,ParentAllocateAdj,ccCost
Code,,,,,,,,,,,
100,MOR,MOR/ R2,"19,077","10,537","6,355","2,000","37,969","-37,969",0,0,0
200,NBR,NBR/ R3,"13,073","2,910","7,328",0,"23,313","-23,313",0,0,0
300,VEL,VEL/ R3,"6,693","1,490","3,752",0,"11,936","-11,936",0,0,0
111,MOR_SM_CUT,0,0,0,0,0,0,"5,681","5,681",0,"5,681"
112,MOR_SM_OTH,0,0,0,0,0,0,"5,681","5,681",0,"5,681"
113,MOR_SM_TUB,0,0,0,0,0,0,"5,681","5,681",0,"5,681"
311,VEL_AS_ASY,0,0,0,0,0,0,"2,387","2,387",0,"2,387"
321,VEL_AL_ALU,0,0,0,0,0,0,"2,387","2,387",0,"2,387"
900,RCPsga,0,0,0,0,0,0,0,0,"51,400","51,400"


In [23]:
#print(f"{buildTaxDf['Total'].sum().round(2)            = }")
#print(f"{buildTaxDf['ccCost'].sum().round(2)           = }")
#print(f"{buildTaxDf['ChildAdjusted'].sum().round(2)    = }")
#print(f"{buildTaxDf['ChildAllocateAdj'].sum().round(2) = }")
#print(f"{buildTaxDf['ParentAllocateAdj'].sum().round(2)= }")

print(f"Sum of Building Taxes:          {buildTaxDf['Total'].sum():,.0f}")
print(f"Sum of Cost Center Cost:        {buildTaxDf['ccCost'].sum():,.0f}")
print(f"Sum of Child  Adjustment Check: {buildTaxDf['ChildAllocateAdj'].sum().round(2)}")
print(f"Sum of Parent Adjustment Chick: {buildTaxDf['ParentAllocateAdj'].sum().round(2)}")

Sum of Building Taxes:          73,219
Sum of Cost Center Cost:        73,219
Sum of Child  Adjustment Check: 0.0
Sum of Parent Adjustment Chick: 0.0


## Other Expense Allocation
**source:**  query performed by Sam Audia.  This was the basis of the 'ExpenseAccountTotalsThroughJuly2021.xlxs' file
- Added codes.
- Stored account to codes in 'expAccountCode.csv'
- Stored inputs to 'Exp_8_30_21.csv'
- Methods for breaking up accounts are given in ExpenseAccountTransactions for accounts 50450 50465 50470 50500

### Methods for expense account allocation:
- Accounts that flow to the products, or to inventory are not used as a basis for the cost center costs
- 50450 (Tooling): Each item was coded.  Totals were allocated based on the calculation.  
- 50465 (Gas): 1/2 to SM CUT; 1/4 to SG&A; 1/4 to AL ALU
- 50470 (Repairs):  1/4 to SM CUT, 3/4 to SG&A
- 50500 (Supplies): approx 33% to SM CUT; 17% to SM OTH; 13% to AS ASY; 35% to AL ALU

In [24]:
expAccountCodeDf = pd.read_csv(SOURCE_FILE_FOLDER +'expAccountCode.csv', converters={'GLAcctNo': lambda x: str(x)})
# stores a link between the expense account General Ledger and thier Codes
expDf = pd.read_csv(SOURCE_FILE_FOLDER +'Exp_8_30_21.csv', converters={'GLAcctNo': lambda x: str(x)})
# see note above.  Contains GLAcctKey, GLAcctNo, Description, Total *** Total is through 7/21 ***
expDf = expDf.set_index('GLAcctNo').join(expAccountCodeDf.set_index('GLAcctNo')).reset_index()
expDf['Total']=expDf['Total'].astype(int)

In [25]:
viewExpDf = addcDescCol(expDf.sort_values(['Code','Total'], ascending=[True,False],ignore_index=True)).copy()
viewExpDf['Total'] = viewExpDf['Total'].apply(lambda x: f'{x:,}')
viewExpDf

,GLAcctNo,GLAcctKey,Description,Total,Code,cDesc
0,520050000,854,Fixed Cap Abs,"350,000",-10,IntMoves
1,599010000,406,Misc Adj COGS,"70,920",-10,IntMoves
2,999980000,834,Mfg Variance,0,-10,IntMoves
3,513700000,832,Abs-Fixed Setup,"-5,970",-10,IntMoves
4,513800000,833,Abs- Var Setup,"-33,759",-10,IntMoves
5,513500000,830,Abs-Fixed Run,"-84,783",-10,IntMoves
6,745150000,857,COS Fix Cap Abs,"-350,000",-10,IntMoves
7,513000000,430,Assembly-Labor Abs,"-381,216",-10,IntMoves
8,513600000,831,Abs-Var Run,"-441,143",-10,IntMoves
9,722000000,343,"City School Tax, Moravia Bldg","7,875",-5,IncludedInTaxSection


In [26]:
viewExpDf.to_csv(SOURCE_FILE_FOLDER +'expTemp.csv')

In [27]:
MONTHS_YTD=7
expCodeDf = expDf.groupby('Code')['Total'].sum().reset_index()
expCodeDf['ExpAnnUnallocated']=12/MONTHS_YTD*expCodeDf['Total']
expCodeDf.set_index('Code',inplace=True)
#addDesc(expCodeDf)

In [28]:
kids = codeChildParentDf['Children'].values
lKids = list(kids[kids != np.array(None)])
flatKidsList = []
for k in lKids: flatKidsList.extend(k)
parents = codeChildParentDf['Parent'].values
lParents = list(parents[parents !=np.array(None)])
lParents = [int(x) for x in lParents]

kidSet = set(flatKidsList)
parentSet = set(lParents)
reqdCodes = kidSet.union(parentSet)

actCodes = set(expCodeDf.index.values)
expCodeDf.reset_index(inplace=True)
for c in reqdCodes:
    if c not in actCodes:
        expCodeDf.loc[len(expCodeDf.index)] = [c, 0, 0]
expCodeDf.set_index('Code', inplace=True)

In [29]:
expCodeDf = allocate(codeChildParentDf, expCodeDf, 'ExpAnnUnallocated', codeAreaDf, codeHeadCountDf)
formatNums(addDesc(expCodeDf).sort_values('Code'))

,Desc,Total,ExpAnnUnallocated,ChildAllocateAdj,ChildAdjusted,ParentAllocateAdj,ccCost
Code,,,,,,,
-10,IntMoves,"-875,951","-1,501,630",0,"-1,501,630",0,"-1,501,630"
-5,IncludedInTaxSection,"41,766","71,598",0,"71,598",0,"71,598"
-2,ContraRevenue,"89,732","153,826",0,"153,826",0,"153,826"
-1,Product,"5,685,413","9,746,422","7,174","9,753,596",0,"9,753,596"
0,RCPall,"252,096","432,164","-432,164",0,0,0
2,ManEmployee,"934,008","1,601,156","-1,601,156",0,0,0
3,ExpTooling,"16,690","28,611","-28,611",0,0,0
4,ExpGas,"43,969","75,375","-75,375",0,0,0
5,ExpRepair,"7,710","13,217","-13,217",0,0,0


In [30]:
#print(f"{expCodeDf['ExpAnnUnallocated'].sum().round(2)= }")
#print(f"{expCodeDf['ccCost'].sum().round(2)           = }")
#print(f"{expCodeDf['ChildAdjusted'].sum().round(2)    = }")
#print(f"{expCodeDf['ChildAllocateAdj'].sum().round(2) = }")
#print(f"{expCodeDf['ParentAllocateAdj'].sum().round(2)= }")

print(f"Sum of Building Taxes:          {expCodeDf['ExpAnnUnallocated'].sum():,.0f}")
print(f"Sum of Cost Center Cost:        {expCodeDf['ccCost'].sum():,.0f}")
print(f"Sum of Child  Adjustment Check: {expCodeDf['ChildAllocateAdj'].sum().round(2)}")
print(f"Sum of Parent Adjustment Chick: {expCodeDf['ParentAllocateAdj'].sum().round(2)}")

Sum of Building Taxes:          13,480,555
Sum of Cost Center Cost:        13,480,555
Sum of Child  Adjustment Check: 0.0
Sum of Parent Adjustment Chick: 0.0


In [31]:
resExpCodeDf = expCodeDf[['ccCost']]
resExpCodeDf.columns=['exp']
resBuildTaxDf = buildTaxDf[['ccCost']]
resBuildTaxDf.columns=['tax']
resDepCodeDf = depCodeDf[['ccCost']]
resDepCodeDf.columns=['dep']
res = resExpCodeDf.join(resBuildTaxDf).join(resDepCodeDf)
res['ccCost']=res['exp']+res['tax']+res['dep']
res.fillna(0, inplace=True)
res = res[res['ccCost']>1]
#addDesc(res).sort_values('Code')

# Work Center Hours and Rates
**source:**  Query described and data in WorkCenterHours.xlxs'

In [32]:
#expAccountCodeDf = pd.read_csv(SOURCE_FILE_FOLDER +'expAccountCode.csv', converters={'GLAcctNo': lambda x: str(x)})
hoursDf = pd.read_csv(SOURCE_FILE_FOLDER +'ccHours.csv')
hoursDf.set_index('Code', inplace=True)
#addDesc(hoursDf)

In [33]:
resHours = res.join(hoursDf)
resHours.fillna(0, inplace=True)
resHours['avgNumPeopleOrMachine']=resHours['AnnHours']/(250*8)
resHours['ccRate'] = resHours['ccCost']/resHours['AnnHours']
resHours['ccRate'] = resHours['ccRate'].apply(lambda x: f"{x:.2f}".format(x))
resHours['avgNumPeopleOrMachine'] = resHours['avgNumPeopleOrMachine'].apply(lambda x: f"{x:.2f}".format(x))
formatNums(addDesc(resHours).sort_values('Code'))

,Desc,exp,tax,dep,ccCost,AnnHours,avgNumPeopleOrMachine,ccRate
Code,,,,,,,,
111,MOR_SM_CUT,"200,244","5,681","266,738","472,664","3,682",1.84,128.34
112,MOR_SM_OTH,"572,548","5,681","137,869","716,098","11,389",5.69,62.87
113,MOR_SM_TUB,"104,176","5,681","49,669","159,527","2,055",1.03,77.61
311,VEL_AS_ASY,"634,813","2,387","19,568","656,769","11,307",5.65,58.08
321,VEL_AL_ALU,"424,376","2,387","189,546","616,310","7,132",3.57,86.41
900,RCPsga,"3,067,003","51,400","129,279","3,247,683",0,0.00,inf


In [34]:
COGSexp = resHours['ccCost'].sum()-resHours.loc[900,'ccCost']
totSales = 15000000
margin = .3
cost = (1-margin)*totSales
print(f"Total Annual COGS Expenses: ${COGSexp:,.0f}")
print(f"Total Annual COGS: ${cost:,.0f}")
print(f"With total sales of {totSales:,.0f} at a margin of {100*margin:.1f}:")
print(f"COGS Expenses are approximately {100*COGSexp/cost:,.0f}% of total COGS costs.")
print(f"COGS Expenses are approximately {100*COGSexp/totSales:,.0f}% of total sales.")

Total Annual COGS Expenses: $2,621,370
Total Annual COGS: $10,500,000
With total sales of 15,000,000 at a margin of 30.0:
COGS Expenses are approximately 25% of total COGS costs.
COGS Expenses are approximately 17% of total sales.


## Discussion of Results
All items produced between 1/1/2021 and 8/1/2021 that we have complete information for (excludes most _ S and _ bulk parts) were costed under two ways were costed under the 'old' and 'new' methods.

**Current** costing method

- Included costs for consumables
- Used initial guesses for rates
- At creation, initial guesses were checked against MAS for resonable agreement for a basket of goods

**Proposed** costing method

- Consumables are included in the burdened rates
- Calculted burdened rates based on allocation of financial data.
    - Assumed what was shipped matches what was made
    - Assumed that data for which costing/ time estimates were missimg matched those for which it existed

Results are shown in **CostComparison_2021_01_01.xlxs**

In [35]:
priceCaptured= 7391397
approxActualSales= 10500000
oldCaptured= 5166834
newCaptured= 5600211

print(f'{priceCaptured=:,.0f}')
print(f'{approxActualSales=:,.0f}')
print(f'{oldCaptured=:,.0f}')
print(f'{newCaptured=:,.0f}')

priceCaptured=7,391,397
approxActualSales=10,500,000
oldCaptured=5,166,834
newCaptured=5,600,211


In [36]:
currentCost= approxActualSales/priceCaptured*oldCaptured*12/7
newCost= approxActualSales/priceCaptured*newCaptured*12/7
price= approxActualSales*12/7
def m(p,c):return (p-c)/p
print("Adjusted for missing values and annualized results summary")
print(f'{newCost - currentCost=:,.0f}')
print(f'Amount Costing Program is Under (Over) Costing per Month: ${(newCost-currentCost)/12:,.0f}')

Adjusted for missing values and annualized results summary
newCost - currentCost=1,055,387
Amount Costing Program is Under (Over) Costing per Month: $87,949


In [37]:
saveAsCsv(formatNums(addDesc(resHours).sort_values('Code')),'ccCosts.csv')

In [38]:
saveAsCsv(addDesc(codeChildParentViewDf).reset_index(),'codeChildParentView.csv')

In [39]:
saveAsCsv(formatNums(addDesc(codeAreaDf)).sort_values('Code').reset_index(),'areaSplit.csv')

In [40]:
saveAsCsv(addDesc(codeHeadCountDf).reset_index(),'headcountSplit.csv')